# Forecast Performance Scorecard

This scorecard provides a systematic, top-down approach to analyze forecast performance and identify areas for model improvement. The process follows a drill-down methodology from high-level business metrics to individual product analysis.

## Scorecard Process Flow:

### 1. **Portfolio Overview** 
Start with the big picture - how is the entire business performing against forecasts? This gives you the overall health of your forecasting system and identifies if there are systematic issues affecting the whole portfolio.

### 2. **Category Performance Analysis**
Drill down into sales categories (Very Low, Low, Alive, Medium, Winning, High Winning) to identify which product segments are driving forecast errors. This helps prioritize which categories need immediate attention.

### 3. **Error Distribution Deep Dive**
For problematic categories, analyze the distribution of forecast errors to understand:
- Are we consistently over-forecasting or under-forecasting?
- What's the magnitude of errors?
- Which products are the biggest contributors to poor performance?

### 4. **Individual Product Investigation**
Focus on the worst-performing products to understand root causes:
- Are there data quality issues?
- Is the product experiencing unusual patterns (promotions, stockouts, seasonality)?
- Are there external factors affecting demand?

### 5. **Actionable Insights Generation**
Based on the analysis, identify specific actions:
- Which products need model retraining?
- What data quality issues need fixing?
- Which categories need different forecasting approaches?
- What external factors should be incorporated into the model?

## How to Use This Scorecard:

1. **Run the complete notebook** to get the full analysis
2. **Start with the business-level view** to understand overall performance
3. **Identify problematic categories** using the error distribution plots
4. **Drill down into specific products** that are driving poor performance
5. **Document findings** walk through lots of skus and identofy dominnat issues then correct. Don't focus on correcting specific edge cases. This will end up messing working cases



# Scorecard

In [1]:
# =============================================================================
# ALL IMPORTS - CONSOLIDATFEVAL IN FIRST CELL
# =============================================================================

# Standard library imports
import sys
import importlib
from importlib import reload

# Third-party imports
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PySpark imports
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window

# Local imports
import configuration as CONFIG
from functions_library.supabase_connection import SupabaseClient
import functions_library.forecast_evaluation as FEVAL

# =============================================================================
# MODULE RELOADING - Ensure latest versions are loaded
# =============================================================================
importlib.reload(CONFIG)
importlib.reload(sys.modules['functions_library.supabase_connection'])
importlib.reload(sys.modules['functions_library.forecast_evaluation'])

# =============================================================================
# SPARK INITIALIZATION
# =============================================================================
spark = SparkSession.builder.appName('ForecastEvaluation').master('local[1]').getOrCreate()
spark.sparkContext.setLogLevel("WARN")

# =============================================================================
# SUPABASE CLIENT INITIALIZATION
# =============================================================================
storage_client = SupabaseClient()

In [2]:
EVALUATION_START_DATE = "2025-10-06"
EVALUATION_END_DATE = "2025-10-14"

In [3]:
products = storage_client.batch_read_table("products", spark)
prod_day_history = storage_client.download_processed_features(spark)
prod_day_forecast = FEVAL.load_latest_per_date_forecasts(storage_client, spark)


In [4]:
evaluation_data = (prod_day_forecast
                   .join(prod_day_history.select("product_id", "date", "sales_units"), on=["product_id", "date"], how="inner")
                   .groupBy("product_id", "style", "age_sales_category", "age_category", "sales_category", "forecast_method")
                   .agg(F.sum("sales_units").alias("sales_units"),
                        F.sum("forecast").alias("forecast")))
evaluation_data = FEVAL.calculate_bias_and_error_bins(evaluation_data)

In [26]:
validation_data = (prod_day_history
                   .join(prod_day_forecast.select("product_id", "date", "forecast_method", "forecast"), on=["product_id", "date"],how="full"))

# Forward fill style column by product_id
window = Window.partitionBy("product_id").orderBy("date")
validation_data = validation_data.withColumn("style", F.last("style", ignorenulls=True).over(window))

validation_data = FEVAL.calculate_bias_and_error_bins(validation_data)

# Business Summary

In [8]:
pdf, plot = FEVAL.plot_business_level(validation_data, EVALUATION_START_DATE=EVALUATION_START_DATE, EVALUATION_END_DATE=EVALUATION_END_DATE)
plot.show()

Evaluation frame (2025-10-06 - 2025-10-14):
  Total Sales: 6761.0
  Total Forecast: 6020.564109669553
  Over/Under Forecast: -740.4358903304474 units (-11.0%)


# Overall Evaluation

In [9]:
# Define per-bin good thresholds (understock / overstock)
GOOD_THRESHOLD = {
    "02| Very Low": (5, 10),
    "03| Low": (5, 10),
    "04| Alive": (5, 20),
    "05| Medium": (5, 20),
    "06| Winning": (5, 20), # Bias
    "07| High Winning": (5, 30) # Bias
}

In [10]:
df, plot = FEVAL.plot_attribute_distribution(evaluation_data, ["sales_category"])
plot.show()


In [11]:
FEVAL.plot_accuracy_distribution(evaluation_data, method="bias").show()
FEVAL.plot_accuracy_distribution(evaluation_data, method="error").show()

In [12]:
df = (evaluation_data
    .where(~(F.col("sales_category").isin(["02| Very Low", "03| Low", "07| High Winning"]))))
FEVAL.plot_accuracy_distribution(df, method="bias").show()

# Investigate Sales Categories

In [40]:
SELECTFEVAL_SALES_BIN = "06| Winning" 
METHOD = "bias"

In [41]:
evaluation_data_category = evaluation_data.where(F.col("sales_category") == SELECTFEVAL_SALES_BIN)
FEVAL.analyze_category_performance(evaluation_data_category)

,product_count,sales_units,forecast,error
0,18,987,1178.561139,191.561139


In [42]:
FEVAL.plot_accuracy_distribution(evaluation_data_category, method=METHOD).show()

# Investigate Products

In [43]:
SELECTFEVAL_BIAS_BIN = "22| > +100"
LIMIT = 10

In [44]:
(evaluation_data_category
    .where(F.col("sales_category") == SELECTFEVAL_SALES_BIN)
    # .where(F.col("bias_bin").rlike(SELECTFEVAL_BIAS_BIN))
    .orderBy(F.desc("error")).limit(LIMIT).toPandas())

,product_id,style,age_sales_category,age_category,sales_category,forecast_method,sales_units,forecast,bias,error,bias_bin,error_bin
0,7ae7f668-15af-4a57-a46e-4d86822a59fa,MWT332,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,prophet,55,106.643052,0.938965,51.643052,21| +90 +100,22| > +50
1,8f295eb3-59c4-4b98-855f-62bea60947d2,MW1556,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,prophet,71,112.468515,0.584064,41.468515,17| +50 +60,20| +40 +45
2,e7fbc959-a027-448f-a3ab-ad57eaa58073,MWT424,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,prophet,64,100.914846,0.576794,36.914846,17| +50 +60,19| +35 +40
3,7a6eb53d-6c3e-48db-90c6-8d4d245d6f10,MWT400,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,prophet,13,42.715847,2.285834,29.715847,22| > +100,17| +25 +30
4,f11424fa-3934-4239-9841-2c29cfd5d8e9,MWT468,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,prophet,42,68.208656,0.624016,26.208656,18| +60 +70,17| +25 +30
5,e9f320d0-8098-461a-813b-8ef9ef4cf7ac,MWT493,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,prophet,32,57.242562,0.788830,25.242562,19| +70 +80,17| +25 +30
6,09796fad-0077-4e4d-b9ec-3f7cf1fb13a3,MW1552,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,prophet,97,119.358014,0.230495,22.358014,14| +20 +30,16| +20 +25
7,b16fa6b6-a5fb-47f0-9b1c-8a679aad50ba,MW1605,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,prophet,65,86.874188,0.336526,21.874188,15| +30 +40,16| +20 +25
8,0d919d89-a662-4f86-91ff-67dce923f820,MW1577,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,prophet,67,75.172916,0.121984,8.172916,13| +10 +20,13| +5 +10
9,4fed30dc-aa28-4fdb-9a67-becbdbcb26fa,MWT428,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,prophet,25,32.212722,0.288509,7.212722,14| +20 +30,13| +5 +10


In [22]:
reload(FEVAL)

<module 'functions_library.forecast_evaluation' from 'c:\\Users\\Cognira\\Desktop\\Aya - Forecast\\functions_library\\forecast_evaluation.py'>

In [47]:
# Example usage
SELECTED_STYLE = "MW1556" 
pdf_product, plot = FEVAL.plot_product_details_daily_short(validation_data, SELECTED_STYLE, products_df=products, EVALUATION_START_DATE=EVALUATION_START_DATE, EVALUATION_END_DATE=EVALUATION_END_DATE)
plot.show()


Product Title: MW1556 - Back Charm | عباية سوداء مطرزة من الجهتين بتفاصيل فاخرة
Price: 252.17
Status: ACTIVE


In [46]:
prod_day_forecast.where(F.col("style") == SELECTED_STYLE).orderBy("date").toPandas()

,date,product_id,age_sales_category,age_category,sales_category,style,forecast_method,forecast,run_id
0,2025-10-07,7ae7f668-15af-4a57-a46e-4d86822a59fa,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,MWT332,prophet,16.397235,1
1,2025-10-08,7ae7f668-15af-4a57-a46e-4d86822a59fa,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,MWT332,prophet,8.951415,1
2,2025-10-09,7ae7f668-15af-4a57-a46e-4d86822a59fa,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,MWT332,prophet,9.046482,1
3,2025-10-10,7ae7f668-15af-4a57-a46e-4d86822a59fa,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,MWT332,prophet,9.704712,1
4,2025-10-11,7ae7f668-15af-4a57-a46e-4d86822a59fa,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,MWT332,prophet,12.623174,1
5,2025-10-12,7ae7f668-15af-4a57-a46e-4d86822a59fa,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,MWT332,prophet,8.919817,1
6,2025-10-13,7ae7f668-15af-4a57-a46e-4d86822a59fa,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,MWT332,prophet,10.844870,1
7,2025-10-14,7ae7f668-15af-4a57-a46e-4d86822a59fa,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,MWT332,prophet,13.720675,2
8,2025-10-15,7ae7f668-15af-4a57-a46e-4d86822a59fa,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,MWT332,prophet,8.767589,2
9,2025-10-16,7ae7f668-15af-4a57-a46e-4d86822a59fa,04| Mature_04| Mature_06| Winning,04| Mature,06| Winning,MWT332,prophet,7.667082,2


In [39]:
# prod_day_history.where(F.col("style") == SELECTED_STYLE).orderBy("date").toPandas()